# Burrows–Wheeler transform检测基因组中重复序列

参考：Li, H., & Durbin, R. (2009). Fast and accurate short read alignment with Burrows–Wheeler transform. bioinformatics, 25(14), 1754-1760.

- Burrows–Wheeler transform
- C(.)
- O(.)
- S(.)

模拟比对（exact matching）

In [1]:
from commons import how_long
# Burrows–Wheeler transform
# 格式与BWA index产出不同

def bwt(seq):
    '''
    Burrows–Wheeler transform
    格式与BWA index产出不同
    '''
    n = len(seq)
    m = [seq[i:n]+seq[0:i] for i in range(n)]
    sort_m = sorted(m)
#     for i,s in enumerate(m):
#         print('{}\t{}'.format(i,s))
#     print('--'*20)
#     for i,s in enumerate(sort_m):
#         print('{}\t{}'.format(i,s))
    B = ''.join([q[-1] for q in sort_m])
    S = list(map(lambda x:m.index(x),sort_m))
    return B,S


## C(.)
def cal_c(seq):
    '''
    C(.)
    '''
    count_a = seq.count('A')
    count_t = seq.count('T')
    count_c = seq.count('C')
    count_g = seq.count('G')
    c_dict = {
        'A':0,
        'C':count_a,
        'G':count_a+count_c,
        'T':count_a+count_c+count_g
    }
    return c_dict

## test C(.)
def cal_c_go(seq):
    '''
    test C(.)
    '''
    count_g = seq.count('g')
    count_o = seq.count('o')
    count_l = seq.count('l')

    c_dict = {
        'g':0,
        'o':count_g+count_l,
        'l':count_g,
    }
    return c_dict

## O(.)
def cal_o(bwt_seq):
    '''
    O(.)
    '''
    O = {}
    alphabet=set(bwt_seq)
    n = len(bwt_seq)
    for i in range(n):
        for alp in alphabet:
            O.setdefault(alp,{})
            O[alp][i]=bwt_seq[0:i+1].count(alp)
    return O

## test exact mapping
def map_on_genome(seq,C,O,S):
    '''
    test exact mapping
    '''
    loc = 0
    n = len(S)
    R_low_aw = 1
    R_top_aw = n-1
    alp_len = len(seq)
    while R_low_aw <= R_top_aw and alp_len > 0:
        alp = seq[alp_len-1]
        R_low_aw = C[alp] + O[alp][R_low_aw - 1] + 1
        R_top_aw = C[alp] + O[alp][R_top_aw]
        alp_len = alp_len-1
#         print(R_low_aw,R_top_aw)
    return S[R_low_aw:R_top_aw+1]

@how_long
def search_rep(rep,C,O,S):
    '''
    search rep
    '''
    loc = 0
    n = len(S)
    R_low_aw = 1
    R_top_aw = n-1
    alp_len = len(rep)
    n_term = 0
    while R_low_aw <= R_top_aw:
        n_term += 1
        final_l = R_low_aw
        final_t = R_top_aw
        for i in range(alp_len):
            alp = rep[alp_len-1-i]
            R_low_aw = C[alp] + O[alp][R_low_aw - 1] + 1
            R_top_aw = C[alp] + O[alp][R_top_aw]
#         print(n_term)
        
#         print(R_low_aw,R_top_aw)
    return S[final_l:final_t+1],(n_term-1)*alp_len

## 模拟数据测试

In [2]:
import random
random.seed(1)
k=100
genome=''.join(random.choices(['A','T','C','G'],k=k)+['$'])
# googol='googol$'
# genome=googol
B,S=bwt(genome)
C=cal_c(genome)
# C=cal_c_go(genome)
O=cal_o(B)

# print('--'*20)
# print('genmoe: {}'.format(genome))
# print('--'*20)
# print('B: {}'.format(B))
# print('--'*20)
# print('S: {}'.format(S))
# print('--'*20)
# print('C: {}'.format(C))
# print('--'*20)
# print('O: {}'.format(O))

### exact mapping

In [3]:
w='TTTCGAA'
locs = map_on_genome(w,C,O,S)
_=list(map(lambda x:print('{}-"{}"-{}'.format(genome[0:x],genome[x:len(w)+x],genome[len(w)+x:len(genome)])),locs))
print('--'*20)
w='AG'
locs=map_on_genome(w,C,O,S)
_=list(map(lambda x:print('{}-"{}"-{}'.format(genome[0:x],genome[x:len(w)+x],genome[len(w)+x:len(genome)])),locs))

AGG-"TTTCGAA"-GTGATCAGGAACGTATAATTAAATTAGCCAGGATCCGTGCTCGGCCAAGTACCCTTCGCTTAACGCTACGGCGACGCTTCGAGGGCGCCT$
----------------------------------------
AGGTTTCGAAGTGATCAGGAACGTATAATTAAATT-"AG"-CCAGGATCCGTGCTCGGCCAAGTACCCTTCGCTTAACGCTACGGCGACGCTTCGAGGGCGCCT$
AGGTTTCGAAGTGATC-"AG"-GAACGTATAATTAAATTAGCCAGGATCCGTGCTCGGCCAAGTACCCTTCGCTTAACGCTACGGCGACGCTTCGAGGGCGCCT$
AGGTTTCGAAGTGATCAGGAACGTATAATTAAATTAGCC-"AG"-GATCCGTGCTCGGCCAAGTACCCTTCGCTTAACGCTACGGCGACGCTTCGAGGGCGCCT$
AGGTTTCGAAGTGATCAGGAACGTATAATTAAATTAGCCAGGATCCGTGCTCGGCCAAGTACCCTTCGCTTAACGCTACGGCGACGCTTCG-"AG"-GGCGCCT$
-"AG"-GTTTCGAAGTGATCAGGAACGTATAATTAAATTAGCCAGGATCCGTGCTCGGCCAAGTACCCTTCGCTTAACGCTACGGCGACGCTTCGAGGGCGCCT$
AGGTTTCGAAGTGATCAGGAACGTATAATTAAATTAGCCAGGATCCGTGCTCGGCCA-"AG"-TACCCTTCGCTTAACGCTACGGCGACGCTTCGAGGGCGCCT$
AGGTTTCGA-"AG"-TGATCAGGAACGTATAATTAAATTAGCCAGGATCCGTGCTCGGCCAAGTACCCTTCGCTTAACGCTACGGCGACGCTTCGAGGGCGCCT$


### rep search

In [4]:
rep='AG'
locs,length=search_rep(rep,C,O,S)
print('locs:{locs}\nlength:{length}'.format(**locals()))

search_rep() cost:0.0112 ms!
locs:[35, 16, 39, 91, 0, 57, 9]
length:2


### rep search 2

人为添加了部分重复

AGGTTTCGAAGTGATC<font color='red' fontsizs=8>ACACACACAC</font>AGGAACGTA

In [5]:
test_genome='AGGTTTCGAAGTGATCAGGAACGTA$'
test_B,test_S=bwt(test_genome)
test_C=cal_c(test_genome)
test_O=cal_o(test_B)

# print('--'*20)
# print('genmoe: {}'.format(test_genome))
# print('--'*20)
# print('B: {}'.format(test_B))
# print('--'*20)
# print('S: {}'.format(test_S))
# print('--'*20)
# print('C: {}'.format(test_C))
# print('--'*20)
# print('O: {}'.format(test_O))

In [6]:
rep='AG'
locs,length=search_rep(rep,test_C,test_O,test_S)
print('locs:{locs}\nlength:{length}'.format(**locals()))
rep='AC'
locs,length=search_rep(rep,test_C,test_O,test_S)
print('locs:{locs}\nlength:{length}'.format(**locals()))

search_rep() cost:0.0110 ms!
locs:[16, 0, 9]
length:2
search_rep() cost:0.0110 ms!
locs:[20]
length:2


### rep search 3

测试10K基因组速度(bwt需要优化，电脑跑不了太大基因组)

In [7]:
from numpy import ceil
random.seed(1)
k=10000
genome=''.join(random.choices(['A','T','C','G'],k=k)+['$'])
B,S=bwt(genome)
C=cal_c(genome)
O=cal_o(B)

with open('../demo_data/test_genome','w') as f:
    for i in range(int(ceil(len(genome)/60))):
        f.write(genome[i*60:(i+1)*60]+'\n')

In [8]:
# 只是测试读取时间
@how_long
def read_file(file):
    with open(file,'r') as f:
        line = f.readline()
        while line:
            line = f.readline()

read_file('../demo_data/test_genome')

read_file() cost:1.3261 ms!


In [9]:
rep = 'AC'
locs,length=search_rep(rep,C,O,S)
print('locs:{locs}\nlength:{length}'.format(**locals()))

search_rep() cost:0.0231 ms!
locs:[8051, 1914, 4141, 3720]
length:6
